<a href="https://colab.research.google.com/github/SindhuBommisetty/smart-recycling-assistant/blob/main/smart_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow matplotlib scikit-learn

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import os

In [ ]:
dataset_path = '/content/dataset-resized.zip'

In [ ]:
import zipfile

zip_file_path = '/content/dataset-resized.zip'
extracted_path = '/content/dataset-resized'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Update the dataset_path to the extracted directory
dataset_path = extracted_path

```
dataset-binary/
├── recyclable/        ← contains cardboard, glass, metal, paper, plastic
└── non-recyclable/    ← contains only trash

In [ ]:
import os
import shutil

# Source folder — your 6-class dataset
src_base = "/content/dataset-resized"

# New binary folders
dest_base = "/content/dataset_binary"
recyclable_classes = {'cardboard', 'glass', 'metal', 'paper', 'plastic'}
non_recyclable_classes = {'trash'}

# Create directories
os.makedirs(os.path.join(dest_base, "recyclable"), exist_ok=True)
os.makedirs(os.path.join(dest_base, "non-recyclable"), exist_ok=True)

# Move or copy files into binary folders
for class_name in os.listdir(src_base):
    class_path = os.path.join(src_base, class_name)
    if not os.path.isdir(class_path):
        continue
    target_folder = "recyclable" if class_name in recyclable_classes else "non-recyclable"
    dest_folder = os.path.join(dest_base, target_folder)

    for filename in os.listdir(class_path):
        src_file = os.path.join(class_path, filename)
        dst_file = os.path.join(dest_folder, f"{class_name}_{filename}")
        shutil.copy(src_file, dst_file)  # or shutil.move if you prefer


In [65]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

binary_dataset_path = "/content/dataset_binary"

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    binary_dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    binary_dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Build a binary classifier
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binary output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_gen, epochs=10, validation_data=val_gen)

# Save model
model.save("recyclable_binary_classifier.h5")

Found 2022 images belonging to 2 classes.
Found 505 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9392 - loss: 0.6483

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


64/64 ━━━━━━━━━━━━━━━━━━━━ 92s 1s/step - accuracy: 0.9393 - loss: 0.6441 - val_accuracy: 0.9465 - val_loss: 0.2291
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.9362 - loss: 0.2036 - val_accuracy: 0.9465 - val_loss: 0.2434
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.9414 - loss: 0.1750 - val_accuracy: 0.9465 - val_loss: 0.2630
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.9483 - loss: 0.1663 - val_accuracy: 0.9485 - val_loss: 0.1524
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - accuracy: 0.9520 - loss: 0.1296 - val_accuracy: 0.9525 - val_loss: 0.1862
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.9564 - loss: 0.1349 - val_accuracy: 0.9426 - val_loss: 0.2835
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - accuracy: 0.9569 - loss: 0.1236 - val_accuracy: 0.9446 - val_loss: 0.2211
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.9532 - loss: 0.1171 - val_accuracy: 0.9228 - val_loss: 0.3250
Epo

In [80]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained binary classifier
model = load_model("recyclable_binary_classifier.h5")

def test_recyclability(img_path):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)[0][0]  # returns value between 0 and 1

    # Output result
    if prediction >= 0.5:
        print(f"♻️ Recyclable (Confidence: {prediction:.2f})")
    else:
        print(f"🗑️ Non-Recyclable (Confidence: {1 - prediction:.2f})")

# Example
test_recyclability("/content/dataset-resized/glass/glass1.jpg")  # Replace with your image path

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
♻️ Recyclable (Confidence: 1.00)


In [81]:
test_recyclability("/content/dataset-resized/trash/trash100.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
🗑️ Non-Recyclable (Confidence: 0.53)


In [82]:
test_recyclability("/content/dataset-resized/cardboard/cardboard100.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
♻️ Recyclable (Confidence: 1.00)


In [84]:
test_recyclability("/content/dataset-resized/trash/trash1.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
🗑️ Non-Recyclable (Confidence: 0.66)
